**importing the dependencies**

In [45]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
#printing the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

**Data pre-processing**

In [4]:

# load the data to a pandas dataframe
import pandas as pd
news_data= pd.read_csv('/content/drive/MyDrive/train.csv')

In [14]:
# first 5 rows of the dataset
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1




* 0--> real news
* 1-->fake news



In [15]:
news_data.shape

(20800, 5)

In [16]:
#checking for missing values
news_data.isnull().sum()


id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [17]:
#replacing missing values with null string
news_data = news_data.fillna('')

In [18]:
#merging the author name and news title
news_data['content'] = news_data['author']+' '+news_data['title']

In [19]:
news_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [20]:
#seperating feature and target
X = news_data.drop(columns='label', axis =1)
Y = news_data['label']

In [21]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [22]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


In [23]:
port_stem = PorterStemmer()

In [24]:
def stemming(content):
  stemmed_content = re.sub('{^a-zA-Z})',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  return stemmed_content

In [25]:
pip install nltk

In [26]:
import nltk
from nltk.stem import PorterStemmer

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [29]:
def stem_text(text):
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in nltk.word_tokenize(text)]
    return ' '.join(stemmed_words)

In [30]:
news_data['content'] = news_data['content'].apply(stem_text)

In [31]:
print(news_data['content'])

0        darrel lucu hou dem aid : we didn ’ t even see...
1        daniel j. flynn flynn : hillari clinton , big ...
2        consortiumnews.com whi the truth might get you...
3        jessica purkiss 15 civilian kill in singl us a...
4        howard portnoy iranian woman jail for fiction ...
                               ...                        
20795    jerom hudson rapper t.i . : trump a ’ poster c...
20796    benjamin hoffman n.f.l . playoff : schedul , m...
20797    michael j. de la merc and rachel abram maci ’ ...
20798    alex ansari nato , russia to hold parallel exe...
20799                david swanson what keep the f-35 aliv
Name: content, Length: 20800, dtype: object


In [32]:
X = news_data['content'].values
Y = news_data['label'].values

In [33]:
print(X)

['darrel lucu hou dem aid : we didn ’ t even see comey ’ s letter until jason chaffetz tweet it'
 'daniel j. flynn flynn : hillari clinton , big woman on campu - breitbart'
 'consortiumnews.com whi the truth might get you fire' ...
 'michael j. de la merc and rachel abram maci ’ s is said to receiv takeov approach by hudson ’ s bay - the new york time'
 'alex ansari nato , russia to hold parallel exerci in balkan'
 'david swanson what keep the f-35 aliv']


In [34]:
print(Y)

[1 0 1 ... 0 1 1]


In [35]:
Y.shape

(20800,)

In [36]:
#converting the textual data to feature vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)


In [37]:
print(X)

  (0, 17368)	0.19174399410123757
  (0, 16828)	0.28346641165391157
  (0, 16528)	0.25381192720384715
  (0, 14268)	0.22848673641548767
  (0, 9588)	0.32258031390091413
  (0, 9314)	0.25917150076241247
  (0, 8372)	0.21989283499386664
  (0, 8281)	0.1557216482531226
  (0, 7657)	0.19440990485341467
  (0, 5573)	0.20699727619147254
  (0, 4572)	0.2654198995473926
  (0, 4358)	0.24001531570779736
  (0, 4161)	0.31929551417430224
  (0, 3501)	0.2189992694442533
  (0, 3025)	0.3261784049022999
  (0, 745)	0.23963204965497484
  (1, 17681)	0.2986181608805123
  (1, 11433)	0.15958152034879636
  (1, 7466)	0.18937399991571108
  (1, 6115)	0.7093432289095839
  (1, 4129)	0.2618965465646962
  (1, 3351)	0.18965266112004633
  (1, 2761)	0.3644003325228251
  (1, 2412)	0.15413615989592025
  (1, 2012)	0.2919368305111939
  :	:
  (20797, 8249)	0.12727671025023768
  (20797, 7697)	0.2114996570384617
  (20797, 4206)	0.20525579557299228
  (20797, 2662)	0.14708542016427884
  (20797, 1794)	0.32539435757302976
  (20797, 1190)	0.2

**splitting the dataset to traing& test data**

In [38]:


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)


**Train the model:logistic regression**

In [41]:
model = LogisticRegression()

In [43]:
model.fit(X_train, Y_train)

LogisticRegression()

**evalution**

accuracy score

In [46]:
#accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy= accuracy_score(X_train_prediction, Y_train)

In [47]:
print('accuracy score of the training data :',training_data_accuracy)

accuracy score of the training data : 0.9869591346153846


In [49]:
#accuracy score on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy= accuracy_score(X_test_prediction, Y_test)

In [50]:
print('accuracy score of the test data :',test_data_accuracy)

accuracy score of the test data : 0.9769230769230769


**Making a predictive system**

In [56]:
X_new = X_test[10]

prediction = model.predict(X_new)
print(prediction)

if prediction[0] == 0:
    print('the news is real')
else:
    print('the news is fake')


[0]
the news is real


In [57]:
print(Y_test[10])

0
